# Finding Foundations: Tracing the Lineages of Academic Knowledge
Jack Beasley (jbeasley), Kristine Guo (kguo98)  
CS 224W Fall 2018  
  
Implementation of methods and evaluation.

In [1]:
import snap
from IPython.display import Image
from collections import defaultdict
from bfs_graph import translate, create_snap_graph, get_bfs_graph
from mag_api import mag_evaluate, mag_get_title, mag_get_date

## Create BFS Subgraph
Retrieve the graph produced by a BFS from the given start paper.

In [ ]:
G, mag_to_snap = get_bfs_graph(2366141641, 2, True, False)
snap_to_mag = {v: k for k, v in mag_to_snap.iteritems()}

In [ ]:
# Sanity check for correct nodes and ID translation.
for node in G.Nodes():
    print node.GetId(), "\t", snap_to_mag[node.GetId()]

In [ ]:
snap.DrawGViz(G, snap.gvlDot, "2366141641.png", "BFS Graph for 2366141641 (Out)")
Image(filename='2366141641.png')

In [ ]:
Graph, mag_to_snap_full = get_bfs_graph(633000, 3, True, True)
snap_to_mag_full = {v: k for k, v in mag_to_snap_full.iteritems()}

In [ ]:
# Sanity check
start_node = Graph.GetNI(mag_to_snap_full[633000])
print("633000 has {} references in full BFS graph".format(start_node.GetOutDeg()))
print("633000 has {} citations in full BFS graph".format(start_node.GetInDeg()))

## MAG Request
Example of MAG API request.

In [ ]:
mag_get_title(2366141641)

## Search Path Count
Implementation of SPC based on Batagelj (2003).

In [40]:
def get_in_degrees(Graph):
    InDegV = snap.TIntPrV()
    snap.GetNodeInDegV(Graph, InDegV)
    return {item.GetVal1() : item.GetVal2() for item in InDegV}

def get_out_degrees(Graph):
    OutDegV = snap.TIntPrV()
    snap.GetNodeOutDegV(Graph, OutDegV)
    return {item.GetVal1() : item.GetVal2() for item in OutDegV}

In [41]:
def compute_n_minus(start, in_degrees, Graph):
    n_minus = defaultdict(int)
    in_deg = in_degrees.copy()
    while len(start) > 0:
        node = start.pop()
        if in_degrees[node] == 0:
            n_minus[node] = 1
        else:
            total = 0
            for citation in Graph.GetNI(node).GetInEdges():
                total += n_minus[citation]
            n_minus[node] = total

        for ref in Graph.GetNI(node).GetOutEdges():
            in_deg[ref] -= 1
            if in_deg[ref] == 0:
                start.add(ref)
    return n_minus

In [42]:
def compute_n_plus(start, out_degrees, Graph):
    n_plus = defaultdict(int)
    out_deg = out_degrees.copy()
    while len(start) > 0:
        node = start.pop()
        if out_degrees[node] == 0:
            n_plus[node] = 1
        else:
            total = 0
            for reference in Graph.GetNI(node).GetOutEdges():
                total += n_plus[reference]
            n_plus[node] = total

        for citation in Graph.GetNI(node).GetInEdges():
            out_deg[citation] -= 1
            if out_deg[citation] == 0:
                start.add(citation)
    return n_plus

In [43]:
def spc(Graph):
    in_degrees = get_in_degrees(Graph)
    out_degrees = get_out_degrees(Graph)
    
    n_minus = compute_n_minus(set([node for node in in_degrees if in_degrees[node] == 0]), in_degrees, Graph)
    n_plus = compute_n_plus(set([node for node in out_degrees if out_degrees[node] == 0]), out_degrees, Graph)
    
    spc_counts = defaultdict(int)
    for edge in Graph.Edges():
        src = edge.GetSrcNId()
        dst = edge.GetDstNId()
        spc_counts[(src, dst)] = n_minus[src] * n_plus[dst]
    return spc_counts

## Edge Betweenness
Call SNAP betweenness function to get edge betweenness of every edge.

In [44]:
def edge_betweenness(Graph):
    Nodes = snap.TIntFltH()
    Edges = snap.TIntPrFltH()
    snap.GetBetweennessCentr(Graph, Nodes, Edges, 1.0, True)
    return {(edge.GetVal1(), edge.GetVal2()): Edges[edge] for edge in Edges}

## Main Path Analysis
Methods for constructing main paths based on different measures.

In [45]:
def mpa(start_node, translate, counts, Graph):
    '''MPA using given counts as edge weights.'''
    node = start_node
    nodes = set([start_node])
    print mag_get_title(translate[node])
    while Graph.GetNI(node).GetOutDeg() != 0:
        next_node = -1
        largest = -1
        for neighbor in Graph.GetNI(node).GetOutEdges():
            if counts[(node, neighbor)] > largest and neighbor not in nodes:
                largest = counts[(node, neighbor)]
                next_node = neighbor
        if next_node < 0:
            break
        node = next_node
        nodes.add(next_node)
        print "-", largest, "->", mag_get_title(translate[node])

In [46]:
def mpa_spc(start_node, translate, Graph):
    '''MPA using SPC as traversal counts.'''
    mpa(start_node, translate, spc(Graph), Graph)

In [47]:
def mpa_bw(start_node, translate, Graph):
    '''MPA using edge betweenness as traversal counts.'''
    mpa(start_node, translate, edge_betweenness(Graph), Graph)

In [48]:
def mpa_val(start_node, translate, values, Graph):
    '''Construct main path by greedily choosing node with highest value.'''
    node = start_node
    nodes = set([start_node])
    print mag_get_title(translate[node])
    while Graph.GetNI(node).GetOutDeg() != 0:
        next_node = -1
        largest = -1
        for neighbor in Graph.GetNI(node).GetOutEdges():
            if values[neighbor] > largest and neighbor not in nodes:
                largest = values[neighbor]
                next_node = neighbor
                nodes.add(neighbor)
        if next_node < 0:
            break
        node = next_node
        print "-", largest, "->", mag_get_title(translate[node])

## Validate Graph
Mitigate cyclic effects by identifying bidirectional edges and removing the edge from the older paper to the newer paper.

In [49]:
def validate_graph(Graph, snap_to_mag):
    bad_edges = set()
    for edge in Graph.Edges():
        src_nid = edge.GetSrcNId()
        dst_nid = edge.GetDstNId()
        if Graph.IsEdge(dst_nid, src_nid):
            first = min(src_nid, dst_nid)
            second = max(src_nid, dst_nid)
            bad_edges.add((first, second))
    for bad_edge in bad_edges:
        date1 = mag_get_date(snap_to_mag[bad_edge[0]])
        date2 = mag_get_date(snap_to_mag[bad_edge[1]])
        if date1 < date2:
            print "Deleting edge (%d, %d)" % (snap_to_mag[bad_edge[1]], snap_to_mag[bad_edge[0]])
            Graph.DelEdge(bad_edge[1], bad_edge[0])
        else:
            print "Deleting edge (%d, %d)" % (snap_to_mag[bad_edge[0]], snap_to_mag[bad_edge[1]])
            Graph.DelEdge(bad_edge[0], bad_edge[1])

## Baseline Methods
Using SNAP functions to get PageRank and node betweenness values for every node in given graph.

### PageRank

In [50]:
def pagerank(paper, snap_to_mag, Graph):
    PRankH = snap.TIntFltH()
    snap.GetPageRank(Graph, PRankH)
    PRankH.SortByDat(False)
    return PRankH

### Node Betweenness

In [56]:
def node_betweenness(snap_to_mag, Graph):
    Nodes = snap.TIntFltH()
    Edges = snap.TIntPrFltH()
    snap.GetBetweennessCentr(Graph, Nodes, Edges, 1.0, True)
    return Nodes

## Compare Results Function
Convenience function to print out results for all evaluated methods.

In [52]:
def compare_results(paper, levels=2, follow_out=True, follow_in=False):
    Graph, mag_to_snap = get_bfs_graph(paper, levels, follow_out, follow_in)
    snap_to_mag = {v: k for k, v in mag_to_snap.iteritems()}
    validate_graph(Graph, snap_to_mag)
    pr = pagerank(mag_to_snap[paper], snap_to_mag, Graph)
    nb = node_betweenness(snap_to_mag, Graph)
    
    print '='*10
    print "MPA (SPC) RESULTS:"
    mpa_spc(mag_to_snap[paper], snap_to_mag, Graph)
    print '='*10
    print "MPA (BETWEENNESS) RESULTS:"
    mpa_bw(mag_to_snap[paper], snap_to_mag, Graph)
    print '='*10
    print "PAGERANK RESULTS:"
    mpa_pr(mag_to_snap[paper], snap_to_mag, pr, Graph)
    print '='*10
    print "NODE BETWEENNESS RESULTS:"
    mpa_pr(mag_to_snap[paper], snap_to_mag, nb, Graph)

## RESULTS
All generated results.

### Node2Vec

In [53]:
compare_results(2366141641, 2, True, False)

{"sourcePaperID":2366141641,"numberBFSLevels":2,"followedOutLinks":true,"followedInLinks":false,"outputEdgeList":"2366141641.txt","numFoundEdges":1684,"numFoundNodes":1395}
MPA (SPC) RESULTS:
node2vec scalable feature learning for networks (2016)
- 2230 -> grarep learning graph representations with global structural information (2015)
- 1010 -> line large scale information network embedding (2015)
- 1336 -> deepwalk online learning of social representations (2014)
- 1710 -> representation learning a review and new perspectives (2013)
- 198 -> a global geometric framework for nonlinear dimensionality reduction (2000)
- 23 -> independent component analysis a new concept (1994)
MPA (BETWEENNESS) RESULTS:
node2vec scalable feature learning for networks (2016)
- 425.366666667 -> community detection in graphs (2010)
- 3.0 -> evolution of networks (2002)
PAGERANK RESULTS:
node2vec scalable feature learning for networks (2016)
- 0.000923760984197 -> nonlinear dimensionality reduction by locall

In [54]:
compare_results(2366141641, 2, True, True)

{"sourcePaperID":2366141641,"numberBFSLevels":2,"followedOutLinks":true,"followedInLinks":true,"outputEdgeList":"2366141641.txt","numFoundEdges":68407,"numFoundNodes":45710}
Deleting edge (2808923352, 2796096336)
Deleting edge (2127048411, 2155369095)
Deleting edge (1989524890, 2127048411)
Deleting edge (2110620844, 2127048411)
Deleting edge (2163922914, 2106439909)
Deleting edge (2584995636, 2595947069)
Deleting edge (2593560573, 1683668173)
Deleting edge (2754629507, 2765574772)
Deleting edge (2127048411, 1942910215)
Deleting edge (2127048411, 2146591355)
Deleting edge (2759045585, 2789220419)
Deleting edge (2700550412, 2614812929)
Deleting edge (1967022297, 2127048411)
Deleting edge (2127048411, 2098873466)
Deleting edge (2808923352, 2809376420)
Deleting edge (2107569009, 1996816151)
Deleting edge (2141599568, 1614298861)
Deleting edge (2008209917, 2127048411)
Deleting edge (2066459332, 2144799688)
Deleting edge (2558460151, 2558748708)
Deleting edge (1557055049, 2135230723)
Deletin

In [57]:
compare_results(2366141641, 3, True, False)

{"sourcePaperID":2366141641,"numberBFSLevels":3,"followedOutLinks":true,"followedInLinks":false,"outputEdgeList":"2366141641.txt","numFoundEdges":38148,"numFoundNodes":19854}
Deleting edge (2137253512, 2605441573)
Deleting edge (2069153192, 2170344111)
Deleting edge (1525961042, 1793121960)
Deleting edge (2614634292, 2163922914)
Deleting edge (2127048411, 2146591355)
Deleting edge (2008209917, 2044988896)
Deleting edge (2050239729, 2068015060)
Deleting edge (2069153192, 2117831564)
Deleting edge (2127048411, 2149274953)
Deleting edge (1967022297, 2127048411)
Deleting edge (2106004777, 2072128103)
Deleting edge (2250968750, 2152722485)
Deleting edge (2107569009, 1996816151)
Deleting edge (2072128103, 2168345951)
Deleting edge (2109715166, 2057885377)
Deleting edge (2137253512, 2136693028)
Deleting edge (2146672645, 2124486835)
Deleting edge (2127048411, 2098873466)
Deleting edge (2156740722, 2136922672)
Deleting edge (2156740722, 2116825644)
Deleting edge (2163922914, 1811734137)
Deleti

### Values Are a Good Thing in Conservation Biology

In [58]:
compare_results(2052309590, 2, True, False)

{"sourcePaperID":2052309590,"numberBFSLevels":2,"followedOutLinks":true,"followedInLinks":false,"outputEdgeList":"2052309590.txt","numFoundEdges":171,"numFoundNodes":164}
MPA (SPC) RESULTS:
values are a good thing in conservation biology (2007)
- 147 -> ecology values and objectivity advancing the debate (2005)
- 109 -> beyond biology toward a more public ecology for conservation (2001)
- 2 -> compass and gyroscope integrating science and politics for the environment (1993)
MPA (BETWEENNESS) RESULTS:
values are a good thing in conservation biology (2007)
- 108.0 -> beyond biology toward a more public ecology for conservation (2001)
- 3.0 -> ecological sustainability as a conservation concept sustentabilidad ecologica como concepto de conservacion (1997)
PAGERANK RESULTS:
values are a good thing in conservation biology (2007)
- 0.00690275328587 -> advocacy and credibility of ecological scientists in resource decisionmaking a regional study (2003)
- 0.00650926342983 -> entering the centu

In [59]:
compare_results(2052309590, 2, True, True)

{"sourcePaperID":2052309590,"numberBFSLevels":2,"followedOutLinks":true,"followedInLinks":true,"outputEdgeList":"2052309590.txt","numFoundEdges":6430,"numFoundNodes":5696}
Deleting edge (1852641894, 1518046545)
MPA (SPC) RESULTS:
values are a good thing in conservation biology (2007)
- 436020 -> ecology values and objectivity advancing the debate (2005)
- 217910 -> beyond biology toward a more public ecology for conservation (2001)
- 11950 -> a science for survival values and conservation biology (1996)
- 8467 -> a sand county almanac (1949)
MPA (BETWEENNESS) RESULTS:
values are a good thing in conservation biology (2007)
- 55617.7424603 -> beyond biology toward a more public ecology for conservation (2001)
- 802.83452381 -> the appearance of ecological systems as a matter of policy (1992)
- 22.8345238095 -> a sand county almanac (1949)
PAGERANK RESULTS:
values are a good thing in conservation biology (2007)
- 0.17837260614 -> a sand county almanac (1949)
NODE BETWEENNESS RESULTS:
valu

In [60]:
compare_results(2052309590, 3, True, False)

{"sourcePaperID":2052309590,"numberBFSLevels":3,"followedOutLinks":true,"followedInLinks":false,"outputEdgeList":"2052309590.txt","numFoundEdges":3460,"numFoundNodes":3065}
Deleting edge (2265645172, 2175653249)
MPA (SPC) RESULTS:
values are a good thing in conservation biology (2007)
- 17023 -> ecology values and objectivity advancing the debate (2005)
- 7346 -> implications of current ecological thinking for biodiversity conservation a review of the salient issues (2005)
- 6414 -> beyond biology toward a more public ecology for conservation (2001)
- 4185 -> the natural imperative for biological conservation (2000)
- 1824 -> current normative concepts in conservation (1999)
- 2197 -> cross scale morphology geometry and dynamics of ecosystems (1992)
- 704 -> large scale management experiments and learning by doing (1990)
- 35 -> adaptive environmental assessment and management (1978)
MPA (BETWEENNESS) RESULTS:
values are a good thing in conservation biology (2007)
- 2175.56904762 -> be

In [61]:
compare_results(2052309590, 4, True, False)

{"sourcePaperID":2052309590,"numberBFSLevels":4,"followedOutLinks":true,"followedInLinks":false,"outputEdgeList":"2052309590.txt","numFoundEdges":40097,"numFoundNodes":28836}
Deleting edge (2126101898, 2147096043)
Deleting edge (2109580099, 2036038071)
Deleting edge (1501126627, 2165702945)
Deleting edge (2164018692, 2160225296)
Deleting edge (2057142259, 1983158799)
Deleting edge (1968468300, 2066187816)
Deleting edge (1964556100, 1992658889)
Deleting edge (2010394253, 1992658889)
Deleting edge (2010394253, 1964556100)
Deleting edge (2132443331, 2041409839)
Deleting edge (2164018692, 2098341261)
Deleting edge (1979909894, 2017380944)
Deleting edge (2121969791, 2164018692)
Deleting edge (2265645172, 2175653249)
MPA (SPC) RESULTS:
values are a good thing in conservation biology (2007)
- 43371700 -> beyond biology toward a more public ecology for conservation (2001)
- 44448588 -> the natural imperative for biological conservation (2000)
- 17619765 -> current normative concepts in conserv

### Network Embedding as Matrix Factorization: Unifying DeepWalk, LINE, PTE, and node2vec

In [62]:
compare_results(2761896323, 2, True, False)

{"sourcePaperID":2761896323,"numberBFSLevels":2,"followedOutLinks":true,"followedInLinks":false,"outputEdgeList":"2761896323.txt","numFoundEdges":1571,"numFoundNodes":1155}
MPA (SPC) RESULTS:
network embedding as matrix factorization unifying deepwalk line pte and node2vec (2018)
- 13278 -> inductive representation learning on large graphs (2017)
- 4567 -> structural deep network embedding (2016)
- 5676 -> grarep learning graph representations with global structural information (2015)
- 13008 -> line large scale information network embedding (2015)
- 24375 -> deepwalk online learning of social representations (2014)
- 22932 -> representation learning a review and new perspectives (2013)
- 104 -> nonlinear dimensionality reduction by locally linear embedding (2000)
MPA (BETWEENNESS) RESULTS:
network embedding as matrix factorization unifying deepwalk line pte and node2vec (2018)
- 233.087301587 -> representation learning a review and new perspectives (2013)
- 22.0 -> modeling pixel mean

In [63]:
compare_results(2761896323, 2, True, True)

{"sourcePaperID":2761896323,"numberBFSLevels":2,"followedOutLinks":true,"followedInLinks":true,"outputEdgeList":"2761896323.txt","numFoundEdges":55686,"numFoundNodes":47035}
Deleting edge (2111002549, 2146591355)
Deleting edge (2767045141, 2624431344)
Deleting edge (2163922914, 2106439909)
Deleting edge (2782733358, 2785662987)
Deleting edge (2260771218, 2744080778)
Deleting edge (2141599568, 1614298861)
Deleting edge (2163922914, 2013035813)
Deleting edge (2510403706, 1495323440)
Deleting edge (2163922914, 1811734137)
Deleting edge (2002276939, 2132914434)
Deleting edge (2118585731, 2109943925)
Deleting edge (2614634292, 2163922914)
Deleting edge (2132914434, 2143420533)
MPA (SPC) RESULTS:
network embedding as matrix factorization unifying deepwalk line pte and node2vec (2018)
- 11421020 -> inductive representation learning on large graphs (2017)
- 21743568 -> community preserving network embedding (2017)
- 6102720 -> node2vec scalable feature learning for networks (2016)
- 29124370 -

In [64]:
compare_results(2761896323, 3, True, False)

{"sourcePaperID":2761896323,"numberBFSLevels":3,"followedOutLinks":true,"followedInLinks":false,"outputEdgeList":"2761896323.txt","numFoundEdges":34040,"numFoundNodes":17866}
Deleting edge (193851967, 2099939455)
Deleting edge (2140833774, 2072128103)
Deleting edge (2066459332, 2144799688)
Deleting edge (2156740722, 2136922672)
Deleting edge (1989368986, 2103829273)
Deleting edge (1916091028, 1979505770)
Deleting edge (2020842694, 1880262756)
Deleting edge (2165966284, 2142623206)
Deleting edge (2153441719, 2164853353)
Deleting edge (1950703464, 2140000690)
Deleting edge (2096091969, 2136676173)
Deleting edge (200434350, 2171009857)
Deleting edge (2109025268, 2097308346)
Deleting edge (2614634292, 2163922914)
Deleting edge (2145038566, 2159291644)
Deleting edge (1989368986, 1866611211)
Deleting edge (2156740722, 2116825644)
Deleting edge (2097308346, 1511160855)
Deleting edge (2072128103, 2168345951)
Deleting edge (2613634265, 2110798204)
Deleting edge (2072128103, 2296073425)
Deleting